In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [24]:
months = ['Gener', 'Febrer', 'Marc', 'Abril', 'Maig', 'Juny', 'Juliol', 'Agost', 'Setembre', 'Octubre', 'Novembre', 'Desembre']
indeces = ['01', '02', '03', '04', '05', '06', '07', '08', '09', '10', '11', '12']
years = ['2020', '2021', '2022', '2023']

In [25]:
def format_data(year):
    data = pd.DataFrame()
    # Cargamos los datos del año correspondiente
    for index, month in zip(indeces, months):
        month_year = pd.read_csv('../raw_data/{}_{}_{}_BicingNou_ESTACIONS.csv'.format(year, index, month))
        data = pd.concat([data, month_year])
    # Nos quedamos con las columnas que nos interesan
    data = data[['station_id', 'last_reported','num_docks_available', 'num_bikes_available']]
    # Formateamos 'last_reported' y creamos las nuevas columnas
    data['last_reported'] = pd.to_datetime(data['last_reported'], unit='s')
    data['year'] = data['last_reported'].dt.year
    data['month'] = data['last_reported'].dt.month
    data['day'] = data['last_reported'].dt.day
    data['hour'] = data['last_reported'].dt.hour

    # Calculamos el % de disponibilidad
    data['percentage_docks_available'] = data['num_docks_available'] / (data['num_bikes_available'] + data['num_docks_available'])

    # Ordenamos por fecha y quitamos los NaNs
    data = data.sort_values('last_reported', ascending = True).dropna(axis = 0)
    data = data.groupby(['station_id', 'year','month','day','hour']).mean().reset_index()[['station_id','year','month','day','hour','percentage_docks_available']]

    # Separamos el dataframe en 5 daframes
    dfs = []
    for i in range(5):
        dfs.append(data[i::5])
        dfs[i].columns = ['station_id','year','month','day','hour','ctx-'+str(4-i)]
        dfs[i]['order_index'] = [i for i in range(len(dfs[i]))]
    data = dfs[4].copy()
    # Reordenamos los datos para ponerlos en el formato que toca
    for i in range(0,4):
        data = pd.merge(data, dfs[i][['order_index', 'ctx-'+str(4-i), ]], right_on = 'order_index', left_on = 'order_index')
    data = data[['station_id', 'year', 'month', 'day', 'hour', 'ctx-4','ctx-3','ctx-2', 'ctx-1', 'ctx-0']]
    data.columns = ['station_id', 'year', 'month', 'day', 'hour', 'ctx-4','ctx-3','ctx-2', 'ctx-1', 'percentage_docks_available']

    return data


In [26]:
for year in years:
    format_data(year).to_csv('data/formated data/data_{}.csv'.format(year), sep=',')

/var/folders/gt/v7xq73cx0nj74cmjgsw8ql_00000gn/T/ipykernel_5726/1063352608.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[i]['order_index'] = [i for i in range(len(dfs[i]))]
/var/folders/gt/v7xq73cx0nj74cmjgsw8ql_00000gn/T/ipykernel_5726/1063352608.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  dfs[i]['order_index'] = [i for i in range(len(dfs[i]))]
/var/folders/gt/v7xq73cx0nj74cmjgsw8ql_00000gn/T/ipykernel_5726/1063352608.py:28: SettingWithCopyWarning: 
A value is trying to be set on a copy